In [3]:
import torch
# import torch.nn as nn
# import torch.optim as optim

from model_factories import get_golem_model, get_resnet18_model, get_resnet34_model, get_vitbase_model
from dataloaders import get_dataloaders
from model_utils import train_epoch, eval, getBestModelParams
from plotter import plot_images

In [4]:
device = torch.device(
        "mps"
        if torch.backends.mps.is_available()
        else "cuda" if torch.cuda.is_available() else "cpu"
    )
train_loader, test_loader, classes = get_dataloaders(test_run=True)
CLASSES_NUM = len(classes)
EPOCHS = 5

## Optimal Model Hyperparameters

In [ ]:
gc = get_golem_model(CLASSES_NUM)
gc.to(device)

lr, loss, criterion, optim = getBestModelParams(gc, train_loader, device)
print(f"Optim lr: {lr}\nLoss: {loss}\nBest optim: {optim}\nBest loss func: {criterion}")

LR = lr

Test for optim <class 'torch.optim.adam.Adam'> and loss func <class 'torch.nn.modules.loss.CrossEntropyLoss'>: 
Learning rate: 0.0001


# Sceanrio 1 - Custom Backbone

In [6]:
gc = get_golem_model(CLASSES_NUM)
gc.to(device)

criterion_gc = criterion()
optimizer_gc = optim(params=gc.parameters(), lr=LR)

In [ ]:
for i in range(3):
    loss = train_epoch(gc, criterion_gc, optimizer_gc, train_loader, device)
    print(f"{i+1}/3: loss={loss}")

In [ ]:
metrics, preds = eval(gc, criterion_gc, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Custom backbone results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)

# Scenario 2 - ResNet18

In [ ]:
gc = get_resnet18_model(CLASSES_NUM)
gc.to(device)

criterion_gc = criterion()
optimizer_gc = optim(params=gc.parameters(), lr=LR)

In [ ]:
for i in range(3):
    loss = train_epoch(gc, criterion_gc, optimizer_gc, train_loader, device)
    print(f"{i+1}/3: loss={loss}")

In [ ]:
metrics, preds = eval(gc, criterion_gc, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Resnet18 backbone results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)

# Scenario 3 - ResNet34

In [18]:
gc = get_resnet34_model(CLASSES_NUM)
gc.to(device)

criterion_gc = criterion()
optimizer_gc = optim(params=gc.parameters(), lr=LR)

In [ ]:
for i in range(3):
    loss = train_epoch(gc, criterion_gc, optimizer_gc, train_loader, device)
    print(f"{i+1}/3: loss={loss}")

In [ ]:
metrics, preds = eval(gc, criterion_gc, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Resnet34 backbone results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)

# Scenario 5

In [15]:
gc = get_vitbase_model(CLASSES_NUM)
gc.to(device)

criterion_gc = criterion()
optimizer_gc = optim(params=gc.parameters(), lr=LR)

In [ ]:
for i in range(3):
    loss = train_epoch(gc, criterion_gc, optimizer_gc, train_loader, device)
    print(f"{i+1}/3: loss={loss}")

In [ ]:
metrics, preds = eval(gc, criterion_gc, test_loader, device)
loss, accuracy, precision, recall, f1 = metrics
print("Vitbase backbone results: ")
print(f"Avg loss {loss}\nAccu {accuracy}\nPrecision {precision}\nRecall {recall}\nF-score {f1}\n")

# plot_images(images, preds, classes)